In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint
from datetime import datetime
import os
import csv

In [2]:
# Save config information.
url = "https://data.ca.gov/api/3/action/datastore_search?resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991&"

# Build partial query URL
limit = 1000
year = 2015
query_url = f"{url}&limit={limit}&q={year}"
response = requests.get(query_url).json()
response

{'help': 'https://data.ca.gov/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 1000,
  'q': '2015',
  'records_format': 'objects',
  'resource_id': 'd73ee828-c2c6-485c-91dc-c26bd9ce3991',
  'total_estimation_threshold': None,
  'records': [{'_id': 2015,
    'OBJECTID': '2015',
    'Year': '2018',
    'State': 'CA',
    'Agency': 'CDF',
    'Unit ID': 'TCU',
    'Fire Name': 'PARROTTS',
    'Local Incident Number': '00009242',
    'Alarm Date': '8/5/2018 12:00:00 AM',
    'Containment Date': '8/20/2018 12:00:00 AM',
    'Cause': '14',
    'Collection Method': '1',
    'Management Objective': '1',
    'GIS Calculated Acres': '135.9872',
    'Comments': None,
    'Complex Name': None,
    'IRWIN ID': '{37FD38AB-7AFB-479F-B968-EDDBC33CD293}',
    'Fire Number (historical use)': None,
    'Complex ID': None,
    'DECADES': '2010',
    'Shape__Area': '890110.71484375',
    'Shape__Length': '7376.47382630404',
    'rank': 0.0573087

In [3]:
df = pd.DataFrame(response['result']['records'])
df_2015 = df.loc[df['Year'] == f'{str(year)}']
columns_to_keep = ['Year', 'Alarm Date','Containment Date','Cause','GIS Calculated Acres']
columns_to_drop = [col for col in df_2015.columns if col not in columns_to_keep]
df_2015 = df_2015.drop(columns=columns_to_drop)
df_2015 = df_2015.dropna (subset = ["Containment Date"])
df_2015.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
1,2015,6/5/2015 12:00:00 AM,6/5/2015 12:00:00 AM,1,7.054897
2,2015,6/11/2015 12:00:00 AM,6/11/2015 12:00:00 AM,14,11.8876
3,2015,12/25/2015 12:00:00 AM,12/29/2015 12:00:00 AM,11,1287.388
4,2015,9/10/2015 12:00:00 AM,9/28/2015 12:00:00 AM,14,70846.53
5,2015,7/2/2015 12:00:00 AM,7/2/2015 12:00:00 AM,7,357.5903


In [4]:
df_2015['Alarm Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2015['Alarm Date']]
df_2015['Containment Date'] = [str(item) for item in df_2015['Containment Date']]
df_2015['Containment Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2015['Containment Date']]
df_2015.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
1,2015,6/5/2015,6/5/2015,1,7.054897
2,2015,6/11/2015,6/11/2015,14,11.8876
3,2015,12/25/2015,12/29/2015,11,1287.388
4,2015,9/10/2015,9/28/2015,14,70846.53
5,2015,7/2/2015,7/2/2015,7,357.5903


In [5]:
df_2015 = df_2015.rename(columns={'Cause': 'Cause #'})
df_2015['Cause'] = ''
df_2015['Cause #'] = df_2015['Cause #'].astype(int)
def map_cause_to_meaning(i):
    if i == 1:
        return 'Lightning'
    elif i == 2:
        return 'Equipment Use'
    elif i == 3:
        return 'Smoking'
    elif i == 4:
        return 'Campfire'
    elif i == 5:
        return 'Debris'
    elif i == 6:
        return 'Railroad'
    elif i == 7:
        return 'Arson'
    elif i == 8:
        return 'Playing with fire'
    elif i == 9:
        return 'Miscellaneous'
    elif i == 10:
        return 'Vehicle'
    elif i == 11:
        return 'Powerline'
    elif i == 12:
        return 'Firefighter Training'
    elif i == 13:
        return 'Non-Firefighter Training'
    elif i == 14:
        return 'Unknown / Unidentified'
    elif i == 15:
        return 'Structure'
    elif i == 16:
        return 'Aircraft'
    elif i == 18:
        return 'Escaped Prescribed Burn'
    elif i == 19:
        return 'Illegal Alien Campfire'
    else:
        return 'N/A'
df_2015['Cause'] = df_2015['Cause #'].apply(map_cause_to_meaning)
df_2015.head()

,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
1,2015,6/5/2015,6/5/2015,1,7.054897,Lightning
2,2015,6/11/2015,6/11/2015,14,11.8876,Unknown / Unidentified
3,2015,12/25/2015,12/29/2015,11,1287.388,Powerline
4,2015,9/10/2015,9/28/2015,14,70846.53,Unknown / Unidentified
5,2015,7/2/2015,7/2/2015,7,357.5903,Arson


In [6]:
df_2015.to_csv("outputs/fires_2015.csv",
                  encoding="utf-8", index=False, header=True)

In [7]:
# Attempt to convert 'Alarm Date' to datetime, coercing errors
df_2015['Alarm Date'] = pd.to_datetime(df_2015['Alarm Date'], errors='coerce')

# Display rows with invalid dates
invalid_dates = df_2015[df_2015['Alarm Date'].isna()]
print(invalid_dates)

# Continue with the processing
df_byalarm = df_2015.sort_values(by='Alarm Date').reset_index()
df_byalarm.head()

Empty DataFrame
Columns: [Year, Alarm Date, Containment Date, Cause #, GIS Calculated Acres, Cause]
Index: []


,index,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,182,2015,2015-01-24,3/1/2015,5,44.545,Debris
1,183,2015,2015-01-24,3/1/2015,5,35.28005,Debris
2,118,2015,2015-02-06,2/13/2015,14,512.4335,Unknown / Unidentified
3,215,2015,2015-02-06,2/6/2015,5,15.84018,Debris
4,7,2015,2015-02-07,2/9/2015,9,6537.519,Miscellaneous


In [8]:
earliest_fire = df_byalarm['Alarm Date'][0]
earliest_fire

Timestamp('2015-01-24 00:00:00')

In [9]:
# Convert 'Containment Date' to datetime, coercing errors
df_2015['Containment Date'] = pd.to_datetime(df_2015['Containment Date'], errors='coerce')

# Drop rows with invalid 'Containment Date'
df_2015_filtered = df_2015.dropna(subset=['Containment Date'])

# Now you can sort and process the DataFrame as needed
df_bycontainment = df_2015_filtered.sort_values(by='Containment Date', ascending=False).reset_index(drop=True)
latest_containment = df_bycontainment['Containment Date'].iloc[0]

df_bycontainment.head()

,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,2015,2015-06-17,2015-12-31,9,31284.44,Miscellaneous
1,2015,2015-12-25,2015-12-29,11,1287.388,Powerline
2,2015,2015-12-25,2015-12-25,14,0.09117375,Unknown / Unidentified
3,2015,2015-12-11,2015-12-11,14,2.801698,Unknown / Unidentified
4,2015,2015-11-11,2015-12-11,14,25.0148,Unknown / Unidentified


In [10]:
latest_containment

Timestamp('2015-12-31 00:00:00')

In [11]:
output_path = os.path.join("outputs", "timeline_dates_2015.csv")
with open(output_path, 'w') as csvfile:

    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')

    # Write the first row (column headers)
    csvwriter.writerow(['Title', 'Date'])
    csvwriter.writerow(['Earliest Fire', earliest_fire])

    # Write the second row
    csvwriter.writerow(['Latest Containment', latest_containment])